# **Clasificador de Frutas con CNN**

1. Preparar el entorno y descargar el dataset

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"gerardodiaz0430","key":"37f5cb85b4c798d79739ad66f64cf0b5"}'}

In [2]:
!pip install -q kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d moltean/fruits

Dataset URL: https://www.kaggle.com/datasets/moltean/fruits
License(s): CC-BY-SA-4.0
100% 3.39G/3.41G [00:25<00:00, 273MB/s]
100% 3.41G/3.41G [00:26<00:00, 140MB/s]


In [4]:
!unzip -q fruits.zip -d fruits

2. Cargar y preparar los datos con ImageDataGenerator

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [6]:
train_dir = 'fruits/fruits-360_100x100/fruits-360/Training'
test_dir = 'fruits/fruits-360_100x100/fruits-360/Test'

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 82296 images belonging to 201 classes.
Found 20494 images belonging to 201 classes.


3. Definir la CNN

In [9]:
from tensorflow.keras import layers, models

In [10]:
num_classes = train_generator.num_classes
print(f"Número de clases detectadas: {num_classes}")

Número de clases detectadas: 201


In [11]:
model = models.Sequential([
    # Primera sección de convolución
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.25),

    # Segunda sección de convolución
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.25),

    # Clasificación
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 98, 98, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 96, 96, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 48, 48, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 48, 48, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 46, 46, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 44, 44, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 30976)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     7,930,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 201)            │        51,657 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,047,337 (30.70 MB)

 Trainable params: 8,047,337 (30.70 MB)

 Non-trainable params: 0 (0.00 B)

4. Entrenamiento del Modelo CNN

In [14]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  12/2572 ━━━━━━━━━━━━━━━━━━━━ 1:15:40 2s/step - accuracy: 0.0026 - loss: 5.5704

KeyboardInterrupt: 

In [ ]:
model.save('modelo_frutas.keras')

In [ ]:
import json

with open('historial_entrenamiento.json', 'w') as f:
    json.dump(history.history, f)

NameError: name 'history' is not defined

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/modelo_frutas.keras')

In [ ]:
import json

with open('/content/drive/MyDrive/historial_entrenamiento.json', 'r') as f:
    history_dict = json.load(f)


5. Visulizacion de Metricas de entrenamiento

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.plot(history_dict['accuracy'], label='Precisión Entrenamiento')
plt.plot(history_dict['val_accuracy'], label='Precisión Validación')
plt.title('Precisión del Modelo')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.grid(True)
plt.show()

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 14 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


FileNotFoundError: [Errno 2] No such file or directory: 'historial_entrenamiento.json'

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(history.history['loss'], label='Pérdida Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida Validación')
plt.title('Pérdida del Modelo')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'plt' is not defined

6. Evaluar el modelo con el conjunto de prueba

In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'fruits/fruits-360/Test',
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Para mantener orden si se comparan predicciones
)


In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'🔍 Precisión en el conjunto de prueba: {test_acc:.4f}')
print(f'📉 Pérdida en el conjunto de prueba: {test_loss:.4f}')

7. Predecir una imagen individual

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Reemplaza 'nombre.jpg' con el nombre real del archivo subido
img_path = list(uploaded.keys())[0]
img = image.load_img(img_path, target_size=(100, 100))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0


In [ ]:
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction)
class_labels = list(train_generator.class_indices.keys())

print(f'🔍 Clase predicha: {class_labels[predicted_class]}')


In [ ]:
plt.imshow(img)
plt.axis('off')
plt.title(f'Predicción: {class_labels[predicted_class]}')
plt.show()
